Configuration de l'environnement


In [2]:
!apt update > /dev/null
!apt install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 38.9 MB/s 


In [4]:
!pip install -q findspark
import findspark
findspark.init()

SparkContext et SparkSession

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
sc = spark.sparkContext

In [7]:
spark

RDD


> Transformation : Ce sont les opérations qui sont appliquées sur un RDD pour créer un nouveau RDD. Filter, groupBy et map sont des exemples de transformations.

> Action : Ce sont les opérations qui sont appliquées sur RDD, qui ordonnent à Spark d'effectuer le calcul et de renvoyer le résultat au pilote.





> Opération de transformation 


> parallelize()

> map(f, preservePartitioning = False) : Un nouveau RDD est renvoyé en appliquant une fonction à chaque élément du RDD. Dans l'exemple suivant, nous formons une paire clé-valeur et mappons chaque chaîne avec une valeur de 1

> join(other, numPartitions = None) : Il renvoie un RDD avec une paire d'éléments avec les clés correspondantes et toutes les valeurs pour cette clé particulière


> cache() : Conserver un RDD avec le niveau de stockage par défaut (MEMORY_ONLY)


> broadcast() : Les variables de diffusion sont utilisées pour enregistrer la copie des données sur tous les nœuds. Cette variable est mise en cache sur toutes les machines et non envoyée sur les machines avec des tâches

> accumulator(n) : Les variables d'accumulateur sont utilisées pour agréger les informations par le biais d'opérations associatives et commutatives. Par exemple, vous pouvez utiliser un accumulateur pour une opération de somme ou des compteurs (dans MapReduce).














In [9]:
#parallelize
liste=["scala", 
   "java", 
   "hadoop", 
   "spark", 
   "akka",
   "spark vs hadoop", 
   "pyspark",
   "pyspark and spark"]
words = sc.parallelize(liste)
print(words)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


In [49]:
words2=sc.broadcast(liste)
print(words2)

Opération d'action


> count(rdd) : Nb elts d'un rdd

> collect(rdd) : Retourne tous les elts d'un rdd

> foreach(f) : Renvoie uniquement les éléments qui remplissent la condition de la fonction à l'intérieur de foreach


> filtre(f) : Un nouveau RDD est renvoyé contenant les élémentsqui remplissent la condition de la fonction à l'intérieur du filtre

> réduire(f) : Effectue une opération arithmétique simple

> is_cached : Permet de vérifier si le RDD est mis en cache ou non

> value : Retourne les elts d'un broadcast ou accumulator

















In [10]:
# count
counts = words.count()
print("Nb elements : {}".format(counts))

Nb elements : 8


In [23]:
#collect
coll = words.collect()
print("Coll : {}".format(coll))

Coll : ['scala', 'java', 'hadoop', 'spark', 'akka', 'spark vs hadoop', 'pyspark', 'pyspark and spark']


In [38]:
#foreach
def f(x): print(x)
fore = words.foreach(f)
print(fore)

None


In [39]:
#filtered
words_filter = words.filter(lambda x: 'spark' in x)
filtered = words_filter.collect()
print("elt filtré : {}".format(filtered))

elt filtré : ['spark', 'spark vs hadoop', 'pyspark', 'pyspark and spark']


In [ ]:
#mapp => A coder
# ormer une paire clé-valeur et mapper chaque chaîne avec une valeur de 1.

In [ ]:
#reduce => A coder
# Générer une liste de valeur entier en 1 et 100
# Créer un rdd de la liste définie ci-dessus
# Calculer la somme des elts du rdd en utilisant la fonction reduce


In [44]:
#join
x = sc.parallelize([("spark", 1), ("hadoop", 4)])
y = sc.parallelize([("spark", 2), ("hadoop", 5)])
joined = x.join(y)
# compléter le code afin d'afficher les elts du rdd joined
print( "Join RDD : {}".format(joined))

Join RDD : PythonRDD[63] at RDD at PythonRDD.scala:53


In [47]:
#cache
words.cache() 
caching = words.persist().is_cached 
print("Words got chached : {}".format(caching))

Words got chached : ParallelCollectionRDD[20] at readRDDFromFile at PythonRDD.scala:274


In [50]:
#broadcast()
data = words2.value 
print("valeur data : {}".format(data) )
elem = words2.value[2] 
print("elt de la postion 2 : {}".format(elem) )

valeur data : ['scala', 'java', 'hadoop', 'spark', 'akka', 'spark vs hadoop', 'pyspark', 'pyspark and spark']
elt de la postion 2 : hadoop


In [57]:
#accumulator
num = sc.accumulator(10) 
def f(x): 
   global num 
   num+=x 
rdd = sc.parallelize([20,30,40,50]) 
rdd.foreach(f) 
final = num.value 
print( "valeur :{}".format(final))

valeur :150


## Exercice 1

### Question 1

In [55]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

sc = spark.sparkContext

lines = sc.textFile("peres-enfants.txt")

groups = lines.map(lambda l: tuple(l.split("\t")))

families = groups.groupByKey().mapValues(tuple)
print(families.collect())

[('ABERNATHY  Ralph', ('ABERNATHY Siegfried', 'ABERNATHY Tobias', 'ABERNATHY Ursala')), ('BARCLAY  Eddie', ('BARCLAY Roseline', 'BARCLAY Vincent', 'BARCLAY Zephan', 'BARCLAY Susann', 'BARCLAY Zabrine', 'BARCLAY Tessie')), ('BARRAULT  Jean-Louis', ('BARRAULT Yonatan', 'BARRAULT Rickie', 'BARRAULT Renaud', 'BARRAULT Tedric')), ('BON  Frédéric', ('BON Yasu', 'BON Rickie')), ('BURNIER  Michel-Antoine', ('BURNIER Tiffany', 'BURNIER Susann')), ('CAMUS  Albert', ('CAMUS Ximena', 'CAMUS Zephan', 'CAMUS Victoria', 'CAMUS Vincent')), ('CHEVERNY  Julien', ('CHEVERNY Simeon',)), ('CHIRAC  Jacques', ('CHIRAC Zack', 'CHIRAC Yves', 'CHIRAC Therese', 'CHIRAC Wolfgang', 'CHIRAC Rickie', 'CHIRAC Yasu', 'CHIRAC Priscilla', 'CHIRAC Zorah ')), ('CLAUDIUS-PETIT  Eugène', ('CLAUDIUS-PETIT Tommy', 'CLAUDIUS-PETIT Solange', 'CLAUDIUS-PETIT Rudolf')), ('CREVEL  René', ('CREVEL Ramona', 'CREVEL Priscilla', 'CREVEL Zackariah', 'CREVEL Ramona')), ('DALADIER  Edouard', ('DALADIER Stephanie', 'DALADIER Yvon', 'DALAD

### Question 2

In [ ]:
format_families = groups.groupByKey().mapValues(len)
format_families.collect()

### Question 3

In [ ]:
groups.groupByKey().mapValues(lambda val: (len(val), tuple(val))).collect()

---


## Exercice 2

### Question 1

In [102]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

sc = spark.sparkContext

def getNotes(line):
  cle, mat, coeff, note = tuple(line.split("\t"))
  return cle, int(note)

fields = [
            "art", "chimie", "bio",  "c2i",
            "économie", "français", "géo", "histoire",
            "informatique", "math", "musique", "philo",
            "physique", "sport", "stat"
            ]

notes = []
for mat in fields:
  notes.append(sc.textFile(f"{mat}.txt").map(getNotes))
  
sc.union(notes).groupByKey().mapValues(lambda values: float(f"{(sum(values) / len(values)):.2f}")).collect()


[('Etudiant 1', 8.4),
 ('Etudiant 2', 6.47),
 ('Etudiant 30', 10.67),
 ('Etudiant 79', 8.93),
 ('Etudiant 86', 9.27),
 ('Etudiant 98', 9.0),
 ('Etudiant 102', 10.07),
 ('Etudiant 106', 10.4),
 ('Etudiant 113', 12.73),
 ('Etudiant 141', 9.07),
 ('Etudiant 142', 13.13),
 ('Etudiant 163', 10.27),
 ('Etudiant 172', 11.33),
 ('Etudiant 175', 9.2),
 ('Etudiant 190', 9.6),
 ('Etudiant 192', 11.6),
 ('Etudiant 204', 9.13),
 ('Etudiant 245', 11.13),
 ('Etudiant 248', 10.6),
 ('Etudiant 270', 10.47),
 ('Etudiant 272', 9.93),
 ('Etudiant 333', 10.2),
 ('Etudiant 357', 7.67),
 ('Etudiant 383', 10.13),
 ('Etudiant 392', 11.67),
 ('Etudiant 399', 9.4),
 ('Etudiant 453', 11.0),
 ('Etudiant 466', 10.13),
 ('Etudiant 475', 9.33),
 ('Etudiant 484', 7.2),
 ('Etudiant 487', 10.8),
 ('Etudiant 489', 9.0),
 ('Etudiant 492', 8.93),
 ('Etudiant 493', 11.47),
 ('Etudiant 497', 11.2),
 ('Etudiant 498', 9.47),
 ('Etudiant 503', 10.73),
 ('Etudiant 516', 11.4),
 ('Etudiant 524', 11.47),
 ('Etudiant 535', 10.47),


### Question 2

In [113]:
def getCoeff(line):
  cle, mat, coeff, note = tuple(line.split("\t"))
  return cle, (int(note), int(coeff))

notes = []
for mat in fields:
  notes.append(sc.textFile(f"{mat}.txt").map(separer_plus_coeff))
  
weighted_notes = sc.union(notes) \
    .groupByKey().mapValues(lambda grouped_notes: float(f"{(sum([note * coef for note, coef in grouped_notes]) / sum([coef for note, coef in grouped_notes])):.2f}")) 

weighted_notes.collect()

[('Etudiant 1', 10.92),
 ('Etudiant 2', 7.38),
 ('Etudiant 3', 17.08),
 ('Etudiant 4', 8.92),
 ('Etudiant 6', 11.0),
 ('Etudiant 12', 9.69),
 ('Etudiant 16', 4.62),
 ('Etudiant 17', 8.38),
 ('Etudiant 19', 6.38),
 ('Etudiant 24', 13.54),
 ('Etudiant 26', 10.85),
 ('Etudiant 28', 5.92),
 ('Etudiant 30', 8.69),
 ('Etudiant 32', 9.54),
 ('Etudiant 43', 11.69),
 ('Etudiant 47', 4.08),
 ('Etudiant 50', 8.77),
 ('Etudiant 55', 7.15),
 ('Etudiant 60', 9.46),
 ('Etudiant 62', 7.69),
 ('Etudiant 67', 9.08),
 ('Etudiant 68', 12.31),
 ('Etudiant 69', 8.08),
 ('Etudiant 71', 4.46),
 ('Etudiant 72', 18.62),
 ('Etudiant 74', 4.69),
 ('Etudiant 76', 6.92),
 ('Etudiant 78', 7.62),
 ('Etudiant 79', 4.46),
 ('Etudiant 81', 3.0),
 ('Etudiant 86', 13.31),
 ('Etudiant 91', 7.69),
 ('Etudiant 92', 8.46),
 ('Etudiant 95', 8.0),
 ('Etudiant 96', 12.46),
 ('Etudiant 98', 14.77),
 ('Etudiant 99', 12.62),
 ('Etudiant 102', 7.85),
 ('Etudiant 105', 1.62),
 ('Etudiant 106', 8.46),
 ('Etudiant 109', 13.69),
 ('Etud

### Question 3

In [139]:
weighted_notes.reduce(lambda a, b: a if a[1] > b[1] else b)

('Etudiant 71170', 20.0)